# Load packages

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.evalModel import evalLogisticModels, getPredDist

# Data

## Load cell lines

In [3]:
trainRNA = pd.read_csv('../../data/processed/RNA_train_cancergenes.csv', index_col=0)
trainCellLines = list(trainRNA.index)

testRNA = pd.read_csv('../../data/processed/RNA_test_cancergenes.csv', index_col=0)
testCellLines = list(testRNA.index)

newRNA = pd.read_csv('../../data/processed/RNA_newcancer_cancergenes.csv', index_col=0)
newCellLines = list(newRNA.index)

## CDR

In [4]:
cdr = pd.read_csv('../../data/processed/drugCellLinePairsData.csv', index_col='DepMap_ID')
trainCDR = cdr.loc[trainCellLines, :].reset_index()
testCDR = cdr.loc[testCellLines, :].reset_index()
newCDR = cdr.loc[newCellLines, :].reset_index()

In [5]:
testTemp = testCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                             'name': 'drug',
                                                                                             'effective': 'true'})

newTemp = newCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                          'name': 'drug',
                                                                                          'effective': 'true'})

# Load drugs

In [6]:
drugs = pd.read_csv('../../data/processed/drug_fingerprints.csv', index_col=0)

drugs.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
cytarabine,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
epinastine,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
floxuridine,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
valrubicin,1,0,0,0,0,1,0,0,1,0,...,0,1,0,1,1,1,0,1,0,0
adapalene,1,1,1,0,0,1,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0


In [7]:
trainDrugs = drugs.loc[list(trainCDR.name.values), :].to_numpy()
testDrugs = drugs.loc[list(testCDR.name.values), :].to_numpy()
newDrugs = drugs.loc[list(newCDR.name.values), :].to_numpy()

trainRNA = trainRNA.loc[list(trainCDR.DepMap_ID.values), :].to_numpy()
testRNA = testRNA.loc[list(testCDR.DepMap_ID.values), :].to_numpy()
newRNA = newRNA.loc[list(newCDR.DepMap_ID.values), :].to_numpy()

In [8]:
trainData = [trainDrugs, trainRNA]
trainEff = trainCDR.effective.to_numpy()
testData = [testDrugs, testRNA]
newData = [newDrugs, newRNA]

In [9]:
del cdr, drugs, trainDrugs, testDrugs, newDrugs, trainRNA, testRNA, newRNA

# Fusion performance

In [10]:
drugPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/drugs/siameseV1/models/'
drugModel = 'DrugFewShot_Layers2_Hidden16_DO0-1_AFrelu_LR0-01_DR0-99_DS1000'
drugPath = os.path.join(drugPath, drugModel)

In [11]:
fusionPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/fusion/embedDrug_rawRNA/models'
bestModel = 'FusionFewShotEmbedDrugRawCell_NL64_64_64_DO0-1_AFrelu_LR0-001_DR0-99_DS1024_BYrna'
fusionPath = os.path.join(fusionPath, bestModel)

In [12]:
evalLog = evalLogisticModels(trainData.copy(), trainEff, testData.copy(), newData.copy(), alt='rf',
                            fusionPath=fusionPath, drugPath=drugPath, rnaPath=None)

In [13]:
testDF, newDF, testWrong, newWrong, testCounts, newCounts = evalLog.evaluate(testDF=testTemp.copy(), 
                                                                            newDF=newTemp.copy(),
                                                                            thresh=0.1)


Average Cell Line precision @ k on test set
	Precision@1: 0.1765
	Precision@2: 0.1961
	Precision@3: 0.183
	Precision@4: 0.1765
	Precision@5: 0.2039
	Precision@10: 0.1872

Average Cell Line precision @ k on newcancer set
	Precision@1: 0.2
	Precision@2: 0.1923
	Precision@3: 0.1949
	Precision@4: 0.1962
	Precision@5: 0.1877
	Precision@10: 0.2157


Test set:
No true effective drugs identified in top 3 for ACH-000012 (top drug: TAS-103)
No true effective drugs identified in top 3 for ACH-000086 (top drug: simvastatin)
No true effective drugs identified in top 3 for ACH-000161 (top drug: entinostat)
No true effective drugs identified in top 3 for ACH-000164 (top drug: 10-hydroxycamptothecin)
No true effective drugs identified in top 3 for ACH-000280 (top drug: STF-118804)
No true effective drugs identified in top 3 for ACH-000305 (top drug: YM-155)
No true effective drugs identified in top 3 for ACH-000316 (top drug: genz-644282)
No true effective drugs identified in top 3 for ACH-000329 (top

In [14]:
testDist = getPredDist(testDF)

Avg varaince of predictions for each drug: 0.0177


In [15]:
newDist = getPredDist(newDF)

Avg varaince of predictions for each drug: 0.0174


In [16]:
testCounts.head()

,1,2,3,total
genz-644282,3,0,0,3
doxorubicin,2,2,0,4
rubitecan,2,1,0,3
XL888,2,1,0,3
TAS-103,2,0,1,3


In [17]:
# poor cell line
testDF[testDF.cell_line == 'ACH-000161'].head(10)

,cell_line,cancer_type,drug,true,pred
6270,ACH-000161,Lung Cancer,entinostat,0,0.622155
6264,ACH-000161,Lung Cancer,XL888,0,0.545948
6236,ACH-000161,Lung Cancer,paclitaxel,0,0.515554
6312,ACH-000161,Lung Cancer,PI3K-IN-2,0,0.508619
6307,ACH-000161,Lung Cancer,beta-lapachone,0,0.499740
6275,ACH-000161,Lung Cancer,FK-866,0,0.496407
6293,ACH-000161,Lung Cancer,nocodazole,0,0.461497
6252,ACH-000161,Lung Cancer,practolol,0,0.443325
6301,ACH-000161,Lung Cancer,echinomycin,0,0.432162
6262,ACH-000161,Lung Cancer,JNJ-26481585,0,0.418852


In [18]:
# poor cell line
testDF[testDF.cell_line == 'ACH-000899'].head(10)

,cell_line,cancer_type,drug,true,pred
4557,ACH-000899,Skin Cancer,LY2606368,0,0.689303
4541,ACH-000899,Skin Cancer,OTS167,0,0.659190
4554,ACH-000899,Skin Cancer,disulfiram,0,0.625135
4545,ACH-000899,Skin Cancer,10-hydroxycamptothecin,1,0.604086
4574,ACH-000899,Skin Cancer,vemurafenib,0,0.557070
4515,ACH-000899,Skin Cancer,JNJ-26481585,0,0.497806
4496,ACH-000899,Skin Cancer,genz-644282,0,0.490567
4526,ACH-000899,Skin Cancer,torasemide,0,0.452455
4503,ACH-000899,Skin Cancer,carmofur,0,0.434582
4560,ACH-000899,Skin Cancer,verubulin,0,0.426944


## Cancer precision

In [19]:
cancerTest, cancerNew = evalLog.getCancerPerformance(testDF.copy(), newDF.copy())

In [20]:
print(round(cancerTest.mean(), 4))
cancerTest

p1    0.1934
p2    0.2193
p3    0.2110
p4    0.1986
p5    0.2164
dtype: float64


,p1,p2,p3,p4,p5
Esophageal Cancer,0.666667,0.333333,0.222222,0.250000,0.333333
Endometrial/Uterine Cancer,0.333333,0.500000,0.555556,0.500000,0.466667
Head and Neck Cancer,0.333333,0.500000,0.444444,0.416667,0.400000
Bladder Cancer,0.333333,0.333333,0.222222,0.250000,0.266667
Ovarian Cancer,0.250000,0.375000,0.416667,0.312500,0.250000
Colon/Colorectal Cancer,0.250000,0.125000,0.166667,0.187500,0.200000
Lung Cancer,0.153846,0.115385,0.076923,0.096154,0.153846
Pancreatic Cancer,0.000000,0.125000,0.166667,0.125000,0.200000
Brain Cancer,0.000000,0.125000,0.083333,0.062500,0.100000
Skin Cancer,0.000000,0.100000,0.066667,0.100000,0.160000


In [21]:
print(round(cancerNew.mean(), 4))
cancerNew

p1    0.2056
p2    0.1983
p3    0.2128
p4    0.2214
p5    0.2208
dtype: float64


,p1,p2,p3,p4,p5
Prostate Cancer,0.500000,0.500000,0.333333,0.500000,0.400000
Bone Cancer,0.333333,0.333333,0.259259,0.222222,0.244444
Neuroblastoma,0.333333,0.333333,0.222222,0.250000,0.200000
Gastric Cancer,0.285714,0.250000,0.261905,0.250000,0.200000
Rhabdoid,0.250000,0.250000,0.250000,0.187500,0.200000
Bile Duct Cancer,0.200000,0.100000,0.133333,0.150000,0.160000
Kidney Cancer,0.153846,0.153846,0.153846,0.153846,0.153846
Thyroid Cancer,0.000000,0.062500,0.125000,0.125000,0.150000
Gallbladder Cancer,0.000000,0.000000,0.333333,0.250000,0.400000
Sarcoma,0.000000,0.000000,0.055556,0.125000,0.100000
